This is our grid search document! Note that there are some keyboard interrupts in our output, because we ran everything again just to make sure it works prior to our final submission! Because the full search takes a long time, we interrupted the kernel. The code works!

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt

In [2]:
# ANN model with 2 hidden layers

print(f"{'~'*15} Building ANN model with 2 hidden layers {'~'*15}\n")

# Load the dataset
df = pd.read_csv("../backend/Datasets/COALINDIA.csv")

# Feature selection and preprocessing
dropped_features = ['Date', 'Symbol', 'Series', 
                    'Trades', 'Turnover', 'Deliverable Volume', 
                    '%Deliverble', 'Last', 'VWAP', 'Prev Close']
df.drop(dropped_features, axis=1, inplace=True)

# Define features (X) and target (Y)
X = df.drop('Close', axis=1)
Y = df['Close']

# Scale the data
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X.values)
Y = scaler_Y.fit_transform(Y.values.reshape(-1, 1))

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=False)

print(f"X_train - {X_train}")
print("*" * 50)
print(f"X_test - {X_test}")
print("*" * 50)

print(f"Y_train - {Y_train}")
print("*" * 50)

print(f"Y_test - {Y_test}")


~~~~~~~~~~~~~~~ Building ANN model with 2 hidden layers ~~~~~~~~~~~~~~~

X_train - [[0.53912913 0.69460631 0.5541304  1.        ]
 [0.69474787 0.72926939 0.71293327 0.06651257]
 [0.72108335 0.72747647 0.6717056  0.09779414]
 ...
 [0.40011971 0.40475123 0.4025042  0.02255257]
 [0.40026934 0.39489018 0.38616583 0.04220904]
 [0.36465659 0.36261766 0.36784242 0.02943177]]
**************************************************
X_test - [[0.36555439 0.37950097 0.37196519 0.01550283]
 [0.37827323 0.38936202 0.38097419 0.01553931]
 [0.37572946 0.37546691 0.37883646 0.00991563]
 ...
 [0.05132426 0.05079934 0.05481753 0.01289286]
 [0.05656142 0.05259226 0.05527561 0.01735301]
 [0.04952865 0.06618856 0.05329058 0.0570686 ]]
**************************************************
Y_train - [[0.69701067]
 [0.71894247]
 [0.66155926]
 ...
 [0.39762656]
 [0.37990086]
 [0.36472886]]
**************************************************
Y_test - [[3.80501728e-01]
 [3.78248460e-01]
 [3.72239748e-01]
 [3.77196936e-01

In [3]:
def create_model_A(activation='relu', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, dropout=0.15):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(hidden, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [4]:
ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for dropout in [0.05, 0.1, 0.15]:
                        # num_batches_train = np.ceil(X_train.shape[0] / batch_size)
                        # num_batches_test = np.ceil(X_test.shape[0] / batch_size)
                        # print(f"Training with batch size: {batch_size}, Number of batches in training: {num_batches_train}, Number of batches in testing: {num_batches_test}")
                        mse, r2 = create_model_A(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, dropout=dropout)
                        ret.append([activation, epochs, batch_size, op_learning_rate, hidden, dropout, mse, r2])
    

17/17 [==============================] - 0s 327us/step
Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.05
Mean Squared Error: 396.23054465541145
R^2: 0.7971158
----------------------------------------------------------------
17/17 [==============================] - 0s 356us/step
Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.1
Mean Squared Error: 620.913439743129
R^2: 0.68207014
----------------------------------------------------------------
17/17 [==============================] - 0s 356us/step
Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.15
Mean Squared Error: 1221.2228351214992
R^2: 0.37469018
----------------------------------------------------------------
17/17 [==============================] - 0s 350us/step
Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 64
Dropout: 0.05
Mean Squared Error: 1562.5630143

17/17 [==============================] - 0s 342us/step
Activation: relu
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Hidden: 64
Dropout: 0.05
Mean Squared Error: 696.9065228786546
R^2: 0.6431589
----------------------------------------------------------------
17/17 [==============================] - 0s 330us/step
Activation: relu
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Hidden: 64
Dropout: 0.1
Mean Squared Error: 83.50387786089452
R^2: 0.957243
----------------------------------------------------------------
17/17 [==============================] - 0s 345us/step
Activation: relu
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Hidden: 64
Dropout: 0.15
Mean Squared Error: 93.09475341246728
R^2: 0.95233214
----------------------------------------------------------------
17/17 [==============================] - 0s 337us/step
Activation: relu
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Hidden: 128
Dropout: 0.05
Mean Squared Error: 567.6875667773

17/17 [==============================] - 0s 344us/step
Activation: relu
Epoch: 50
Batch Size: 40
Optimizer learning rate: 1e-05
Hidden: 128
Dropout: 0.05
Mean Squared Error: 207.91602616965108
R^2: 0.89353955
----------------------------------------------------------------
17/17 [==============================] - 0s 339us/step
Activation: relu
Epoch: 50
Batch Size: 40
Optimizer learning rate: 1e-05
Hidden: 128
Dropout: 0.1
Mean Squared Error: 344.1809232583253
R^2: 0.82376707
----------------------------------------------------------------
17/17 [==============================] - 0s 339us/step
Activation: relu
Epoch: 50
Batch Size: 40
Optimizer learning rate: 1e-05
Hidden: 128
Dropout: 0.15
Mean Squared Error: 394.8835023703364
R^2: 0.79780555
----------------------------------------------------------------
17/17 [==============================] - 0s 348us/step
Activation: relu
Epoch: 50
Batch Size: 40
Optimizer learning rate: 0.0001
Hidden: 32
Dropout: 0.05
Mean Squared Error: 67.2252

17/17 [==============================] - 0s 338us/step
Activation: relu
Epoch: 75
Batch Size: 24
Optimizer learning rate: 0.0001
Hidden: 32
Dropout: 0.05
Mean Squared Error: 186.82231358078388
R^2: 0.90434027
----------------------------------------------------------------
17/17 [==============================] - 0s 335us/step
Activation: relu
Epoch: 75
Batch Size: 24
Optimizer learning rate: 0.0001
Hidden: 32
Dropout: 0.1
Mean Squared Error: 728.1865990942192
R^2: 0.6271424
----------------------------------------------------------------
17/17 [==============================] - 0s 354us/step
Activation: relu
Epoch: 75
Batch Size: 24
Optimizer learning rate: 0.0001
Hidden: 32
Dropout: 0.15
Mean Squared Error: 870.2502541406369
R^2: 0.5544007
----------------------------------------------------------------
17/17 [==============================] - 0s 361us/step
Activation: relu
Epoch: 75
Batch Size: 24
Optimizer learning rate: 0.0001
Hidden: 64
Dropout: 0.05
Mean Squared Error: 14.162320

17/17 [==============================] - 0s 352us/step
Activation: relu
Epoch: 75
Batch Size: 32
Optimizer learning rate: 0.0001
Hidden: 64
Dropout: 0.05
Mean Squared Error: 106.34448611084746
R^2: 0.9455478
----------------------------------------------------------------
17/17 [==============================] - 0s 346us/step
Activation: relu
Epoch: 75
Batch Size: 32
Optimizer learning rate: 0.0001
Hidden: 64
Dropout: 0.1
Mean Squared Error: 238.53485819100382
R^2: 0.8778616
----------------------------------------------------------------
17/17 [==============================] - 0s 334us/step
Activation: relu
Epoch: 75
Batch Size: 32
Optimizer learning rate: 0.0001
Hidden: 64
Dropout: 0.15
Mean Squared Error: 317.50434023774454
R^2: 0.8374264
----------------------------------------------------------------
17/17 [==============================] - 0s 332us/step
Activation: relu
Epoch: 75
Batch Size: 32
Optimizer learning rate: 0.0001
Hidden: 128
Dropout: 0.05
Mean Squared Error: 153.529

17/17 [==============================] - 0s 326us/step
Activation: relu
Epoch: 75
Batch Size: 40
Optimizer learning rate: 0.0001
Hidden: 128
Dropout: 0.05
Mean Squared Error: 87.6117017703376
R^2: 0.9551397
----------------------------------------------------------------
17/17 [==============================] - 0s 318us/step
Activation: relu
Epoch: 75
Batch Size: 40
Optimizer learning rate: 0.0001
Hidden: 128
Dropout: 0.1
Mean Squared Error: 125.39901988356131
R^2: 0.9357912
----------------------------------------------------------------
17/17 [==============================] - 0s 338us/step
Activation: relu
Epoch: 75
Batch Size: 40
Optimizer learning rate: 0.0001
Hidden: 128
Dropout: 0.15
Mean Squared Error: 64.75636009795689
R^2: 0.9668424
----------------------------------------------------------------
17/17 [==============================] - 0s 357us/step
Activation: relu
Epoch: 75
Batch Size: 40
Optimizer learning rate: 0.001
Hidden: 32
Dropout: 0.05
Mean Squared Error: 2434.3454

17/17 [==============================] - 0s 355us/step
Activation: relu
Epoch: 100
Batch Size: 24
Optimizer learning rate: 0.001
Hidden: 32
Dropout: 0.05
Mean Squared Error: 4180.510382172136
R^2: -1.1405711
----------------------------------------------------------------
17/17 [==============================] - 0s 342us/step
Activation: relu
Epoch: 100
Batch Size: 24
Optimizer learning rate: 0.001
Hidden: 32
Dropout: 0.1
Mean Squared Error: 4650.804618280276
R^2: -1.3813782
----------------------------------------------------------------
17/17 [==============================] - 0s 362us/step
Activation: relu
Epoch: 100
Batch Size: 24
Optimizer learning rate: 0.001
Hidden: 32
Dropout: 0.15
Mean Squared Error: 4800.4387937924785
R^2: -1.4579964
----------------------------------------------------------------
17/17 [==============================] - 0s 362us/step
Activation: relu
Epoch: 100
Batch Size: 24
Optimizer learning rate: 0.001
Hidden: 64
Dropout: 0.05
Mean Squared Error: 3088.21

17/17 [==============================] - 0s 345us/step
Activation: relu
Epoch: 100
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 64
Dropout: 0.05
Mean Squared Error: 3685.8283274371884
R^2: -0.88727605
----------------------------------------------------------------
17/17 [==============================] - 0s 340us/step
Activation: relu
Epoch: 100
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 64
Dropout: 0.1
Mean Squared Error: 4136.327574837826
R^2: -1.1179473
----------------------------------------------------------------
17/17 [==============================] - 0s 322us/step
Activation: relu
Epoch: 100
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 64
Dropout: 0.15
Mean Squared Error: 4468.127905448637
R^2: -1.2878413
----------------------------------------------------------------
17/17 [==============================] - 0s 332us/step
Activation: relu
Epoch: 100
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 128
Dropout: 0.05
Mean Squared Error: 3663.

InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/mean_squared_error/Reshape_1 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/nayeelimtiaz/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/nayeelimtiaz/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/nayeelimtiaz/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/nayeelimtiaz/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/nayeelimtiaz/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Users/nayeelimtiaz/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/Users/nayeelimtiaz/anaconda3/lib/python3.11/asyncio/events.py"

In [ ]:
print("Best MSE for ANN:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for ANN:", sorted(ret, key=lambda x:-x[-1])[0])

In [ ]:
def create_model_P(activation=None, epochs=100, batch_size=32, op_learning_rate=1e-5):
    model = Sequential()
    model.add(Dense(1, input_dim=X_train.shape[1], activation=activation))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    # Calculate MSE
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error (Perceptron): {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2


In [ ]:
ret = []
for activation in [None, 'relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                
                # num_batches_train = np.ceil(X_train.shape[0] / batch_size)
                # num_batches_test = np.ceil(X_test.shape[0] / batch_size)
                # print(f"Training with batch size: {batch_size}, Number of batches in training: {num_batches_train}, Number of batches in testing: {num_batches_test}")
                mse, r2 = create_model_P(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate)
                ret.append([activation, epochs, batch_size, op_learning_rate, mse, r2])
    

In [ ]:
print("Best MSE for Perceptron:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for Perceptron:", sorted(ret, key=lambda x:-x[-1])[0])

In [ ]:
def create_model_D(activation='tanh', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, layers=11, dropout=0.1):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    # Building DNN model with 11 hidden layers
    model = Sequential()

    # Input layer
    # Input layer will have same number of neurons as number of feature variables
    model.add(Dense(64, input_dim=X_train.shape[1], activation=activation))

    # Hidden layers
    # Play around with number of neurons in each hidden layer.
    # Too many neurons leads to overcomplexity, not enough means too simple
    # Tanh activation function here is used b/c it is recommended to use
    # when there are more hidden layers.
    for _ in range(layers-1):
        model.add(Dense(hidden, activation=activation))
        model.add(Dropout(dropout)) # This helps with preventing overfitting

    # Output layer
    # Output layer will have 1 neuron b/c there's only 1 target variable
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nLayers: {layers}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [ ]:
m, r = create_model_D()

In [ ]:
# GRID SEARCH

ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for layers in [6, 9, 11]:
                        for dropout in [0.05, 0.1, 0.15]:
                            mse, r2 = create_model_D(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, layers=layers, dropout=dropout)
                            ret.append([activation, epochs, batch_size, op_learning_rate, hidden, layers, dropout, mse, r2])
    

In [ ]:
print("Best MSE for DNN:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for DNN:", sorted(ret, key=lambda x:-x[-1])[0])